In [20]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sqlite3
connection = sqlite3.connect("./database/bdfinal.sql")

In [27]:
columns = "o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_3pm,\
    d_3pa,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_pts,tmORB,tmDRB,tmTRB,opptmORB,opptmDRB,opptmTRB,won,lost,GP,homeW,homeL,\
        awayW,awayL,confW,confL,min,attend,firstRound,semis,finals"

query_test = "SELECT year, playoff, seeded, " + columns + " FROM Teams WHERE year = 10"
query_train = "SELECT year, playoff, seeded, " + columns + " FROM Teams WHERE year <> 10"

train_data = pd.read_sql(query_train,connection)
pd.get_dummies(train_data,)


train_data['firstRound'] = train_data.firstRound.astype('category')
train_data['semis'] = train_data.semis.astype('category')
train_data['finals'] = train_data.finals.astype('category')
train_inputs = train_data.loc[:, train_data.columns != "playoff"].values
train_labels = train_data["playoff"].values

test_data = pd.read_sql(query_test,connection)
test_data['firstRound'] = test_data.firstRound.astype('category')
test_data['semis'] = test_data.semis.astype('category')
test_data['finals'] = test_data.finals.astype('category')
test_inputs = test_data.loc[:, test_data.columns != "playoff"].values
test_labels = test_data["playoff"].values

print(train_inputs)
print(train_labels)
print(test_inputs)
print(test_labels)


[[9 0 895 ... '' '' '']
 [1 0 812 ... '' '' '']
 [2 0 746 ... 'W' 'W' 'L']
 ...
 [7 0 1016 ... 'L' '' '']
 [8 0 877 ... '' '' '']
 [9 0 885 ... '' '' '']]
['N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y'
 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y'
 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N'
 'N' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'Y' 'Y'
 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N'
 'Y' 'N' 'N']
[[10 0 1089 2428 569 755 114 374 404 855 1259 547 741 329 590 121 2861
  996 2363 624 807 181 530 353 821 1174 615 700 347 601 133 2797 0 0 0 0
  0 0 18 16 34 12 5 6 11 10 12 6950 120737 'L' '' '']
 [10 0 930 2136 527 693 186 471 307 776 1083 532 605 249 532 138 2573
  1004 2269 479 613 206 593 359 797 1156 610 636 273

In [28]:
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train_inputs,train_labels)
clf.score(test_inputs,test_labels)

ValueError: could not convert string to float: ''